# Click clustering
In this notebook we plan to cluster the students in each course by their click activity. This is to see if we can create clusters that might belong more strongly to pass or fail. 

In [83]:
import pandas as pd
import numpy as np
import glob

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

from sklearn.linear_model import LogisticRegression

from collections import Counter

import math

In [5]:
import matplotlib.pyplot as plt

# Load data into panda-frames and inspect

In [6]:
filenames = glob.glob("../data/*")

In [7]:
dataframes = []
for i, filename in enumerate(filenames): 
    print(i, filename.split("/")[-1])
    df = pd.read_csv(filename)
    dataframes.append(df)
    display(df)

0 data\assessments.csv


,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0
3,AAA,2013J,1755,TMA,166.0,20.0
4,AAA,2013J,1756,TMA,215.0,30.0
...,...,...,...,...,...,...
201,GGG,2014J,37443,CMA,229.0,0.0
202,GGG,2014J,37435,TMA,61.0,0.0
203,GGG,2014J,37436,TMA,124.0,0.0
204,GGG,2014J,37437,TMA,173.0,0.0


1 data\courses.csv


,code_module,code_presentation,module_presentation_length
0,AAA,2013J,268
1,AAA,2014J,269
2,BBB,2013J,268
3,BBB,2014J,262
4,BBB,2013B,240
5,BBB,2014B,234
6,CCC,2014J,269
7,CCC,2014B,241
8,DDD,2013J,261
9,DDD,2014J,262


2 data\studentAssessment.csv


...

,code_module,code_presentation,id_student,date_registration,date_unregistration
0,AAA,2013J,11391,-159.0,NaN
1,AAA,2013J,28400,-53.0,NaN
2,AAA,2013J,30268,-92.0,12.0
3,AAA,2013J,31604,-52.0,NaN
4,AAA,2013J,32885,-176.0,NaN
...,...,...,...,...,...
32588,GGG,2014J,2640965,-4.0,NaN
32589,GGG,2014J,2645731,-23.0,NaN
32590,GGG,2014J,2648187,-129.0,NaN
32591,GGG,2014J,2679821,-49.0,101.0


5 data\studentVle.csv


,code_module,code_presentation,id_student,id_site,date,sum_click
0,AAA,2013J,28400,546652,-10,4
1,AAA,2013J,28400,546652,-10,1
2,AAA,2013J,28400,546652,-10,1
3,AAA,2013J,28400,546614,-10,11
4,AAA,2013J,28400,546714,-10,1
...,...,...,...,...,...,...
10655275,GGG,2014J,675811,896943,269,3
10655276,GGG,2014J,675578,896943,269,1
10655277,GGG,2014J,654064,896943,269,3
10655278,GGG,2014J,654064,896939,269,1


6 data\vle.csv


,id_site,code_module,code_presentation,activity_type,week_from,week_to
0,546943,AAA,2013J,resource,NaN,NaN
1,546712,AAA,2013J,oucontent,NaN,NaN
2,546998,AAA,2013J,resource,NaN,NaN
3,546888,AAA,2013J,url,NaN,NaN
4,547035,AAA,2013J,resource,NaN,NaN
...,...,...,...,...,...,...
6359,897063,GGG,2014J,resource,NaN,NaN
6360,897109,GGG,2014J,resource,NaN,NaN
6361,896965,GGG,2014J,oucontent,NaN,NaN
6362,897060,GGG,2014J,resource,NaN,NaN


In [12]:
studentAssesment_df = dataframes[2]
studentInfo_df = dataframes[3]
courses_df = dataframes[1]
assesments_df = dataframes[0]
studentVle_df = dataframes[5]
vle_df = dataframes[6]
studentRegistration_df = dataframes[4]

In [13]:
studentInfo_df['result_encoded'] = studentInfo_df['final_result'].map({'Pass':1, 'Distinction':1, 'Withdrawn':0, 'Fail':0})

## Merge pandas frame to get data-format

In [14]:
# first join studentVLE with vle to get full view of vle activity
merged_vle_df = pd.merge(studentVle_df, vle_df, on = ['id_site', 'code_module', 'code_presentation'], how='left')

In [15]:
merged_vle_df

,code_module,code_presentation,id_student,id_site,date,sum_click,activity_type,week_from,week_to
0,AAA,2013J,28400,546652,-10,4,forumng,NaN,NaN
1,AAA,2013J,28400,546652,-10,1,forumng,NaN,NaN
2,AAA,2013J,28400,546652,-10,1,forumng,NaN,NaN
3,AAA,2013J,28400,546614,-10,11,homepage,NaN,NaN
4,AAA,2013J,28400,546714,-10,1,oucontent,NaN,NaN
...,...,...,...,...,...,...,...,...,...
10655275,GGG,2014J,675811,896943,269,3,homepage,NaN,NaN
10655276,GGG,2014J,675578,896943,269,1,homepage,NaN,NaN
10655277,GGG,2014J,654064,896943,269,3,homepage,NaN,NaN
10655278,GGG,2014J,654064,896939,269,1,forumng,NaN,NaN


## One- course data

In [84]:
# set mudule and presentation
code_module = 'GGG'
code_presentation = '2014J'

In [93]:
def logistic_regression_result_prediction(code_module, code_presentation): 
    # extract only vle stats from the given course
    course_vle_df = merged_vle_df[(merged_vle_df.code_module==code_module)&(merged_vle_df.code_presentation==code_presentation)]

    # set index to id_student for easier extraction based on indivisual students later
    course_vle_df = course_vle_df.set_index('id_student')

    # extract student information for course module
    course_studentInfo_df = studentInfo_df.loc[(studentInfo_df.code_module==code_module)&(studentInfo_df.code_presentation==code_presentation), :]
    
    # count the proportion of different classes and print
    cnt_final_results = Counter(list(course_studentInfo_df['final_result']))
    print(cnt_final_results)
    
    # extrect activity types
    activities = list(set(course_vle_df['activity_type']))

    # extract day-information
    days = sorted(list(set(course_vle_df['date'])))
    first_day, last_day = days[0], days[-1]
    duration = last_day-first_day
    periods = 1
    period_len = math.ceil(duration/periods)

    def get_click_matrix(student_id, course_vle_df=course_vle_df, activities=activities, period_len=period_len, first_day=first_day, periods=periods): 
        oneStudentVle = course_vle_df.loc[student_id, :]
        click_matrix = np.zeros((len(activities), periods))

        # insert clicks
        for i, vle_row in oneStudentVle.iterrows(): 
            # extract matrix index
            resource_index = activities.index(vle_row['activity_type'])
            period_index = int((vle_row['date']-first_day)/(period_len+1))
            # add clicks to right cell
            click_matrix[resource_index, period_index] += vle_row['sum_click']

        return click_matrix

    def get_pass_fail(student_id, course_studentInfo_df=course_studentInfo_df): 
        one_hot_result = np.zeros(3)
        result_encoded = int(course_studentInfo_df.loc[course_studentInfo_df['id_student']==student_id, 'result_encoded'])

        return result_encoded

    # format x and y data for model training 
    student_ids = list(set(course_vle_df.index))
    x, y = [], []
    for student_id in student_ids: 
        try: 
            click_matrix = get_click_matrix(student_id)
            result_encoded = get_pass_fail(student_id)
            x.append(click_matrix)
            y.append(result_encoded)
        except Exception as e: 
            pass
            #print(student_id)
            #print(e)
    X = np.array(x)
    y = np.array(y)
    
    # reshape X to be one dimensional input datapoints
    X = np.reshape(X, (-1, X.shape[1]*X.shape[2]))

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 3)
    
    # print the counted y_train and y test
    cnt_binary = Counter(list(y_train))
    print("Training data: ", cnt_binary)
    cnt_binary = Counter(list(y_test))
    print("testing data: ", cnt_binary)

    # fit model
    clf = LogisticRegression(random_state=0).fit(X_train, y_train)

    y_pred = clf.predict(X_test)

    print(classification_report(y_test, y_pred))
    
    return 0

## Iterate through multiple courses

In [94]:
# set mudule and presentation
chosen_courses = [('AAA', '2013J'), ('BBB', '2013J'), ('CCC', '2014J'), ('DDD', '2013J'), ('GGG', '2014J')]

for chosen_course in chosen_courses: 
    code_module = chosen_course[0]
    code_presentation = chosen_course[1]

    print(f"Code module: {code_module}")
    print(f"Code presentation: {code_presentation}")

    logistic_regression_result_prediction(code_module, code_presentation)
    print()

Code module: AAA
Code presentation: 2013J
Counter({'Pass': 258, 'Withdrawn': 60, 'Fail': 45, 'Distinction': 20})
Training data:  Counter({1: 226, 0: 76})
testing data:  Counter({1: 52, 0: 24})
              precision    recall  f1-score   support

           0       0.50      0.04      0.08        24
           1       0.69      0.98      0.81        52

    accuracy                           0.68        76
   macro avg       0.59      0.51      0.44        76
weighted avg       0.63      0.68      0.58        76


Code module: BBB
Code presentation: 2013J


C:\Users\perni\miniconda3\envs\tf_env\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Counter({'Pass': 896, 'Withdrawn': 644, 'Fail': 521, 'Distinction': 176})
Training data:  Counter({1: 859, 0: 626})
testing data:  Counter({1: 213, 0: 159})
              precision    recall  f1-score   support

           0       0.89      0.79      0.84       159
           1       0.86      0.93      0.89       213

    accuracy                           0.87       372
   macro avg       0.88      0.86      0.87       372
weighted avg       0.87      0.87      0.87       372


Code module: CCC
Code presentation: 2014J


C:\Users\perni\miniconda3\envs\tf_env\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Counter({'Withdrawn': 1077, 'Pass': 709, 'Fail': 406, 'Distinction': 306})
Training data:  Counter({0: 1023, 1: 793})
testing data:  Counter({0: 234, 1: 221})
              precision    recall  f1-score   support

           0       0.81      0.95      0.87       234
           1       0.94      0.76      0.84       221

    accuracy                           0.86       455
   macro avg       0.87      0.86      0.86       455
weighted avg       0.87      0.86      0.86       455


Code module: DDD
Code presentation: 2013J


C:\Users\perni\miniconda3\envs\tf_env\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Counter({'Pass': 731, 'Withdrawn': 681, 'Fail': 428, 'Distinction': 98})
Training data:  Counter({0: 747, 1: 658})
testing data:  Counter({0: 181, 1: 171})
              precision    recall  f1-score   support

           0       0.79      0.86      0.82       181
           1       0.83      0.75      0.79       171

    accuracy                           0.81       352
   macro avg       0.81      0.81      0.81       352
weighted avg       0.81      0.81      0.81       352


Code module: GGG
Code presentation: 2014J


C:\Users\perni\miniconda3\envs\tf_env\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Counter({'Pass': 317, 'Fail': 179, 'Distinction': 127, 'Withdrawn': 126})
Training data:  Counter({1: 350, 0: 205})
testing data:  Counter({1: 94, 0: 45})
              precision    recall  f1-score   support

           0       0.89      0.73      0.80        45
           1       0.88      0.96      0.92        94

    accuracy                           0.88       139
   macro avg       0.89      0.85      0.86       139
weighted avg       0.89      0.88      0.88       139




## Iterate through all the courses

In [95]:
for index, row in courses_df.iterrows(): 
    code_module = row['code_module']
    code_presentation = row['code_presentation']
    
    print(f"Code module: {code_module}")
    print(f"Code presentation: {code_presentation}")

    logistic_regression_result_prediction(code_module, code_presentation)
    print()

Code module: AAA
Code presentation: 2013J
Counter({'Pass': 258, 'Withdrawn': 60, 'Fail': 45, 'Distinction': 20})
Training data:  Counter({1: 226, 0: 76})
testing data:  Counter({1: 52, 0: 24})
              precision    recall  f1-score   support

           0       0.50      0.04      0.08        24
           1       0.69      0.98      0.81        52

    accuracy                           0.68        76
   macro avg       0.59      0.51      0.44        76
weighted avg       0.63      0.68      0.58        76


Code module: AAA
Code presentation: 2014J


C:\Users\perni\miniconda3\envs\tf_env\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Counter({'Pass': 229, 'Withdrawn': 66, 'Fail': 46, 'Distinction': 24})
Training data:  Counter({1: 203, 0: 81})
testing data:  Counter({1: 50, 0: 21})
              precision    recall  f1-score   support

           0       0.50      0.05      0.09        21
           1       0.71      0.98      0.82        50

    accuracy                           0.70        71
   macro avg       0.61      0.51      0.46        71
weighted avg       0.65      0.70      0.61        71


Code module: BBB
Code presentation: 2013J


C:\Users\perni\miniconda3\envs\tf_env\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Counter({'Pass': 896, 'Withdrawn': 644, 'Fail': 521, 'Distinction': 176})
Training data:  Counter({1: 859, 0: 626})
testing data:  Counter({1: 213, 0: 159})
              precision    recall  f1-score   support

           0       0.89      0.79      0.84       159
           1       0.86      0.93      0.89       213

    accuracy                           0.87       372
   macro avg       0.88      0.86      0.87       372
weighted avg       0.87      0.87      0.87       372


Code module: BBB
Code presentation: 2014J


...

Counter({'Pass': 664, 'Fail': 421, 'Withdrawn': 411, 'Distinction': 118})
Training data:  Counter({1: 622, 0: 583})
testing data:  Counter({1: 160, 0: 142})
              precision    recall  f1-score   support

           0       0.90      0.72      0.80       142
           1       0.79      0.93      0.85       160

    accuracy                           0.83       302
   macro avg       0.85      0.82      0.83       302
weighted avg       0.84      0.83      0.83       302


Code module: FFF
Code presentation: 2014B


C:\Users\perni\miniconda3\envs\tf_env\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Counter({'Pass': 547, 'Withdrawn': 462, 'Fail': 384, 'Distinction': 107})
Training data:  Counter({0: 557, 1: 527})
testing data:  Counter({0: 145, 1: 127})
              precision    recall  f1-score   support

           0       0.93      0.85      0.89       145
           1       0.84      0.93      0.88       127

    accuracy                           0.89       272
   macro avg       0.89      0.89      0.89       272
weighted avg       0.89      0.89      0.89       272


Code module: GGG
Code presentation: 2013J


C:\Users\perni\miniconda3\envs\tf_env\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Counter({'Pass': 451, 'Fail': 294, 'Distinction': 141, 'Withdrawn': 66})
Training data:  Counter({1: 470, 0: 243})
testing data:  Counter({1: 122, 0: 57})
              precision    recall  f1-score   support

           0       0.92      0.81      0.86        57
           1       0.91      0.97      0.94       122

    accuracy                           0.92       179
   macro avg       0.92      0.89      0.90       179
weighted avg       0.92      0.92      0.91       179


Code module: GGG
Code presentation: 2014J
Counter({'Pass': 317, 'Fail': 179, 'Distinction': 127, 'Withdrawn': 126})
Training data:  Counter({1: 350, 0: 205})
testing data:  Counter({1: 94, 0: 45})
              precision    recall  f1-score   support

           0       0.89      0.73      0.80        45
           1       0.88      0.96      0.92        94

    accuracy                           0.88       139
   macro avg       0.89      0.85      0.86       139
weighted avg       0.89      0.88      0.88     